<a href="https://colab.research.google.com/github/TanmayBj23/AI_LAB/blob/main/1BM22CS303_Week8_FOL_ForwardReasoning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# Define the Knowledge Base (KB)
kb = []
# Facts
kb.append(("fact", "American", ["Robert"]))
kb.append(("fact", "Enemy", ["A", "America"]))
kb.append(("fact", "Missile", ["T1"]))
# Rules
kb.append(("rule", ["Missile(x)"], "Weapon(x)"))  # All missiles are weapons
kb.append(("rule", ["Weapon(x)", "Sells(Robert, x, A)", "Hostile(A)"], "Criminal(Robert)"))  # Selling weapons to hostiles implies criminality
kb.append(("rule", ["Enemy(x, America)"], "Hostile(x)"))  # Enemies of America are hostile
kb.append(("fact", "Sells", ["Robert", "T1", "A"]))  # Robert sells T1 to A

# Forward Reasoning Function
def forward_reasoning(kb, query):
    inferred = set()  # Already inferred facts
    while True:
        new_inferred = set()
        for entry in kb:
            if entry[0] == "fact":
                # Add atomic facts to inferred
                fact_name = entry[1]
                fact_args = tuple(entry[2])
                new_inferred.add((fact_name, fact_args))

            elif entry[0] == "rule":
                # Check if the rule's premise is satisfied
                premises = entry[1]
                conclusion = entry[2]

                # Evaluate premises
                premises_satisfied = True
                substitutions = {}
                for premise in premises:
                    satisfied = False
                    for fact in inferred:
                        fact_name, fact_args = fact
                        if match(premise, fact_name, fact_args, substitutions):
                            satisfied = True
                            break
                    if not satisfied:
                        premises_satisfied = False
                        break

                # If all premises are satisfied, infer the conclusion
                if premises_satisfied:
                    conclusion_name, conclusion_args = parse_predicate(conclusion)
                    conclusion_args = [
                        substitutions.get(arg, arg) for arg in conclusion_args
                    ]
                    new_inferred.add((conclusion_name, tuple(conclusion_args)))

        # If no new facts are inferred, stop
        if not new_inferred.difference(inferred):
            break
        inferred.update(new_inferred)

        # Check if the query is inferred
        query_name, query_args = parse_predicate(query)
        if (query_name, tuple(query_args)) in inferred:
            return True

    return False

# Helper functions for matching and parsing
def match(premise, fact_name, fact_args, substitutions):
    premise_name, premise_args = parse_predicate(premise)
    if premise_name != fact_name or len(premise_args) != len(fact_args):
        return False
    for p_arg, f_arg in zip(premise_args, fact_args):
        if p_arg.islower():  # Variable
            substitutions[p_arg] = f_arg
        elif p_arg != f_arg:  # Constant mismatch
            return False
    return True

def parse_predicate(predicate):
    name, args = predicate.split("(")
    args = args[:-1].split(",")  # Remove closing ")"
    return name, args

# Query
query = "Criminal(Robert)"

# Check if the query can be proved
if forward_reasoning(kb, query):
    print(f"The query {query} can be proved.")
else:
    print(f"The query {query} cannot be proved.")


The query Criminal(Robert) cannot be proved.
